In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader
# from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
# from llama_index.text_splitter import SentenceSplitter
import logging

logging.basicConfig(level=logging.INFO)

In [39]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")

In [9]:
# Path to your directory containing text files
DATA_DIR = "/home/kris/dev/syenza-docs/pdfs-tests"

# Load the first 5 documents
documents_full = SimpleDirectoryReader(DATA_DIR).load_data()[:3]

In [10]:
documents_full

[Document(id_='29623de4-3d2c-41ec-9bba-a9a56849204b', embedding=None, metadata={'page_label': '1', 'file_name': '2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_path': '/home/kris/dev/syenza-docs/pdfs-tests/2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_type': 'application/pdf', 'file_size': 448851, 'creation_date': '2024-07-11', 'last_modified_date': '2024-05-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Cite this article as: Kuo C-C, Chang H-H, Hsing C-H, Hii H-P, Wu N-C, Hsu C-M et al . Robotic mitral valve replacements with bioprosthetic valves in 52 patients:\nexperience from a tertiary referral hospital. Eur J Cardiothorac Surg 2018; doi:10.1093/ejcts/ezy134.

In [6]:
# Path to your directory containing text files
DATA_DIR = "/home/kris/dev/syenza-docs/pdfs-tests"

# Load the first 5 documents
documents = SimpleDirectoryReader(DATA_DIR).load_data()

In [7]:
documents

[Document(id_='8e6da2f4-ae0f-4a2a-9518-adf35271ab74', embedding=None, metadata={'page_label': '1', 'file_name': '2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_path': '/home/kris/dev/syenza-docs/pdfs-tests/2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_type': 'application/pdf', 'file_size': 448851, 'creation_date': '2024-07-11', 'last_modified_date': '2024-05-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Cite this article as: Kuo C-C, Chang H-H, Hsing C-H, Hii H-P, Wu N-C, Hsu C-M et al . Robotic mitral valve replacements with bioprosthetic valves in 52 patients:\nexperience from a tertiary referral hospital. Eur J Cardiothorac Surg 2018; doi:10.1093/ejcts/ezy134.

In [14]:
print(len(documents))
for doc in documents:
    print(len(doc.text))

26
5468
5327
5213
3892
6088
7176
4627
4335
4422
3731
5258
1753
3209
3247
4134
2726
4755
3070
5474
5039
3350
4524
2782
3666
3725
5619


In [18]:
# Path to your directory containing text files
DATA_DIR = "/home/kris/dev/syenza-docs/pdfs-tests"

# Load the first 5 documents
documents_with_docs = SimpleDirectoryReader(DATA_DIR).load_data()

In [19]:
len(documents_with_docs)
for doc in documents_with_docs:
    print(len(doc.text))

5468
5327
5213
3892
6088
7176
4627
4335
4422
3731
5258
1753
3209
3247
4134
2726
4755
3070
5474
5039
3350
4524
2782
3666
3725
5619
37270
61516


Important:
- PDFs are split into single pages - number of documents = number of pages (per file)
- Docs are entire documents. 1 Document per file (no matter how many pages)

Splitting into nodes

In [20]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)

index = VectorStoreIndex.from_documents(documents_with_docs, transformations=[text_splitter])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [21]:
index

In [22]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=[text_splitter])

nodes = pipeline.run(documents=documents_with_docs)

In [47]:
documents

[Document(id_='8e6da2f4-ae0f-4a2a-9518-adf35271ab74', embedding=None, metadata={'page_label': '1', 'file_name': '2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_path': '/home/kris/dev/syenza-docs/pdfs-tests/2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_type': 'application/pdf', 'file_size': 448851, 'creation_date': '2024-07-11', 'last_modified_date': '2024-05-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Cite this article as: Kuo C-C, Chang H-H, Hsing C-H, Hii H-P, Wu N-C, Hsu C-M et al . Robotic mitral valve replacements with bioprosthetic valves in 52 patients:\nexperience from a tertiary referral hospital. Eur J Cardiothorac Surg 2018; doi:10.1093/ejcts/ezy134.

In [24]:
len(nodes)

150

Checking overlap:

In [44]:
for node in nodes[:5]:
    print("Start:", node.text[:100])
    print("End:", node.text[-100:])
    print("-*"*25)

Start: Cite this article as: Kuo C-C, Chang H-H, Hsing C-H, Hii H-P, Wu N-C, Hsu C-M et al . Robotic mitral
End: e replacement (MVR) emerged in the late 1990s as an alternative approach to conventional sternotomy.
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Start: With the increased use of bioprosthetic valves worldwide and strong patient desire for minimally inv
End: term results are encouraging. Both aortic
cross-clamp and CPB times can be improved with experience.
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Start: CONCLUSIONS: Robotic MVR with bioprosthetic valves is safe, feasible and reproducible. Mid-term resu
End: lished by Oxford University Press on behalf of the European Association for Cardio-Thoracic Surgery.
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Start: [11]
†These authors contributed equally to this work.
CONVENTIONAL
VALVE OPERATIONS
VCThe Author(s) 
End: ic.oup.com/ejcts/advance-article-abstract/doi/10.1093/ejcts/ezy134/4956919
by guest
on 12 Ap

In [51]:
for node in nodes[:5]:
    print(node.text)
    print("---")

Cite this article as: Kuo C-C, Chang H-H, Hsing C-H, Hii H-P, Wu N-C, Hsu C-M et al . Robotic mitral valve replacements with bioprosthetic valves in 52 patients:
experience from a tertiary referral hospital. Eur J Cardiothorac Surg 2018; doi:10.1093/ejcts/ezy134.
Robotic mitral valve replacements with bioprosthetic valves in
52 patients: experience from a tertiary referral hospital
Chia-Cheng Kuoa,b,c, †, Hsiao-Huang Changb,d,†, Chung-Hsi Hsinge,f, Hiong-Ping Hiia,N a n - C h u nW ua,
Chin-Ming Hsua, Chun-I Chenaand Bor-Chih Chenga,g,*
aDivision of Cardiovascular Surgery, Department of Surgery, Chi Mei Medical Center, Tainan, Taiwan
bDivision of Cardiovascular Surgery, Department of Surgery, Taipei Veteran General Hospital, Taipei, Taiwan
cDepartment of Surgery, School of Medicine, National Defense Medical Center, Taipei, Taiwan
dDepartment of Surgery, School of Medicine, Taipei Medical University, Taipei, Taiwan
eDepartment of Anesthesiology, Chi Mei Medical Center, Tainan, Taiwan
fDe

In [41]:
for node in nodes:
    tkn_count = len(encoding.encode(node.text))
    if tkn_count > 500:
        print(tkn_count)
        print(node.text)
    # print("--"*25)

In [48]:
new_nodes = text_splitter.get_nodes_from_documents(documents[:5])

In [53]:
for node in new_nodes[5:10]:
    print(node.text)
    print("---")

Thehospital’s institutional review board approved the study protocol.
Surgical technique
After the induction of general anaesthesia, patients were intu-bated smoothly as usual. A central venous catheter and a 6.5-Frintroducer sheath were placed into the right internal jugular vein.Patients were positioned in the supine position with the rightchest elevated approximately 30
/C14and the right arm tucked at the
side. External deﬁbrillator pads were placed properly. A multi-plane transoesophageal echocardiography probe was insertedbefore incision to conﬁrm the severity of the valvular heart dis-
ease. A 2-cm oblique right groin incision was made for peripheralcannulation. After systemic heparinization, cardiopulmonary by-pass (CPB) was established under transoesophageal echocardiog-raphy guidance. The patient’s femoral artery was cannulated forsystemic retrograde perfusion. Adjunctive distal femoral perfu-sion was achieved using an 8-Fr arterial cannula (Medtronic,Minneapolis, MN, USA). Bi

In [49]:
new_nodes

[TextNode(id_='ef0fea26-9ed6-4572-8bc0-eb21db9aa039', embedding=None, metadata={'page_label': '1', 'file_name': '2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_path': '/home/kris/dev/syenza-docs/pdfs-tests/2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_type': 'application/pdf', 'file_size': 448851, 'creation_date': '2024-07-11', 'last_modified_date': '2024-05-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8e6da2f4-ae0f-4a2a-9518-adf35271ab74', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': '2018 Robotic MVR with bioprosthetic valves in 52 patients - KUO EJCTS_ezy134.pdf', 'file_path': 

In [31]:
text_splitter256 = SentenceSplitter(chunk_size=256, chunk_overlap=25)

pipeline256 = IngestionPipeline(transformations=[text_splitter256])

nodes256 = pipeline256.run(documents=documents_with_docs)

In [32]:
len(nodes256)

310

In [33]:
for node in nodes256:
    print(len(node.text))

625
812
760
621
510
600
823
876
296
891
870
808
758
551
713
723
253
716
726
738
913
791
609
461
532
629
622
626
739
537
490
371
635
645
650
717
706
851
685
527
387
532
722
861
833
738
791
778
932
792
533
415
533
549
563
527
564
594
541
537
501
763
781
963
907
753
535
635
971
825
803
884
353
687
863
840
849
588
743
884
699
951
759
714
501
516
401
426
381
147
703
883
454
675
427
171
636
816
556
379
586
408
316
998
1071
961
455
534
852
684
451
579
368
767
841
798
955
668
695
330
636
675
661
699
664
586
932
985
948
769
765
713
291
731
748
711
853
950
628
426
875
614
775
742
435
745
563
682
475
587
380
340
396
552
869
725
675
655
568
740
669
704
542
576
724
715
545
934
803
909
640
599
677
703
733
579
600
475
646
883
830
753
598
647
637
758
963
846
903
394
959
813
919
661
787
988
506
770
736
810
729
780
680
812
912
682
713
646
640
742
549
847
719
755
901
786
638
674
531
650
777
841
806
904
735
656
912
602
709
863
825
467
715
913
1002
849
917
704
838
825
854
857
511
907
975
700
645
873
760
61

In [38]:
for node in nodes256:
    tkn_count = len(encoding.encode(node.text))
    if tkn_count > 250:
        print(tkn_count)
        print(node.text)

Conclusions

The SentenceSplitter is based on number of tokens.

So:
- `chunk_size` and `chunk_overlap` are in tokens